In [1]:
import os
import rasterio
import cv2 as cv
import sys

sys.path.insert(1, "../")
from utils import *
from skimage.metrics import structural_similarity as ssim

* ECC works for images of the same type with very good overlap. not working with images with small overlaps (not coorectly registering for multi-sensor cases at least without pre-processing)
* very very slow! (for original images)
* Phase correlation is basically global co-registration in AROSICS. not working for transformation. It only does translation shifts but it is very fast. It can support multi-sensor as it could work on images with partial overlaps.
* It seems that optical flow is doing better than phase correlaton but the improvement is trivial. Also it is very fast.

In [2]:
DATA_DIR = "../data/files_list"

S1_PRODUCTS = ["SLC", "GRD"]
S2_PRODUCTS = ["L1C", "L2A"]

s1_au_df = pd.read_csv(os.path.join(DATA_DIR, "s1_au.csv"), names=["ID", "Path"])
s1_an_df = pd.read_csv(os.path.join(DATA_DIR, "s1_an.csv"), names=["ID", "Path"])

s2_au_df = pd.read_csv(os.path.join(DATA_DIR, "s2_au.csv"), names=["ID", "Path"])
s2_an_df = pd.read_csv(os.path.join(DATA_DIR, "s2_an.csv"), names=["ID", "Path"])

s1_au_slc_dict = get_scenes_dict(s1_au_df, ["SLC"])
s1_au_grd_dict = get_scenes_dict(s1_au_df, ["GRD"])

s1_an_slc_dict = get_scenes_dict(s1_an_df, ["SLC"])
s1_an_grd_dict = get_scenes_dict(s1_an_df, ["GRD"])

s2_au_l1c_dict = get_scenes_dict(s2_au_df, ["L1C"], False)
s2_au_l2a_dict = get_scenes_dict(s2_au_df, ["L2A"], False)

s2_an_l1c_dict = get_scenes_dict(s2_an_df, ["L1C"], False)
s2_an_l2a_dict = get_scenes_dict(s2_an_df, ["L2A"], False)

#### Coregister arbitrary scenes

In [ ]:
s2_an_l1c_r031_dict = get_scenes_dict(s2_an_df, ["L1C", "R031"], False)
id = list(s2_an_l1c_r031_dict.keys())[0]
tdf = s2_an_df[s2_an_df.ID == id]
tdf = tdf[tdf.Path.apply(lambda x: "L1C" in x)]
tdf = tdf[tdf.Path.apply(lambda x: "N0511_R031" in x)]
s2_an_l1c_secne_files = list(tdf.Path)
ref_id = 4
ref_scene = s2_an_l1c_secne_files[ref_id]

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)
with ZipFile(ref_scene) as f:
    f.extractall(f"../data/inputs/{id}/ref")
ref_image_dir = os.path.join("../data/inputs/", id, "ref")

ref_tci_files = list(
    filter(
        lambda f: "TCI" in f,
        glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True),
    )
)
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

print(f"Reference image: {ref_image}")

params = {
    "translation_x": 5.0,
    "translation_y": 10.0,
    "rotation_angle": 2.5,
    "scale": 1.0,
}

tgt_image_dir = os.path.join("../data/inputs/", id, "tgt")
os.makedirs(tgt_image_dir, exist_ok=True)

tgt_image = os.path.join(tgt_image_dir, "tgt.tif")
warp_affine_dataset(ref_image, tgt_image, **params)

_, (axb, axt) = plt.subplots(1, 2, figsize=(10, 20))
show(downsample_dataset(ref_image, 0.1)[0], ax=axb, title="Reference scene")
show(downsample_dataset(tgt_image, 0.1)[0], ax=axt, title="Target scene")

In [126]:
ref_image_ds = os.path.join(ref_image_dir, "ref_ds.tif")
downsample_dataset(ref_image, 0.1, ref_image_ds)
tgt_image_ds = os.path.join(tgt_image_dir, "tgt_ds.tif")
_ = downsample_dataset(tgt_image, 0.1, tgt_image_ds)

In [127]:
ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_image).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img = cv.cvtColor(
    flip_img(rasterio.open(tgt_image).read().copy()), cv.COLOR_BGR2GRAY
)
ref_img_ds = cv.cvtColor(
    flip_img(rasterio.open(ref_image_ds).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img_ds = cv.cvtColor(
    flip_img(rasterio.open(tgt_image_ds).read().copy()), cv.COLOR_BGR2GRAY
)

##### ECC

In [28]:
warp_mode = cv.MOTION_HOMOGRAPHY
warp_matrix = np.eye(3, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_img_ds, tgt_img_ds, warp_matrix, warp_mode, criteria
)

tgt_aligned_homography_ds = cv.warpPerspective(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)
tgt_aligned_homography = cv.warpPerspective(
    tgt_img,
    warp_matrix,
    (tgt_img.shape[1], tgt_img.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [29]:
warp_mode = cv.MOTION_AFFINE
warp_matrix = np.eye(2, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_img_ds, tgt_img_ds, warp_matrix, warp_mode, criteria
)

tgt_aligned_affine_ds = cv.warpAffine(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)
tgt_aligned_affine = cv.warpAffine(
    tgt_img,
    warp_matrix,
    (tgt_img.shape[1], tgt_img.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 4, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_aligned_homography_ds, cmap="jet")
axes[2].set_xlabel(
    f"Registered (hompgraphy), ssim:{np.round(ssim(ref_img_ds, tgt_aligned_homography_ds), 3)}"
)
axes[3].imshow(tgt_aligned_affine_ds, cmap="jet")
axes[3].set_xlabel(
    f"Registered (affine), ssim:{np.round(ssim(ref_img_ds, tgt_aligned_affine_ds), 3)}"
)

In [ ]:
_, axes = plt.subplots(1, 4, figsize=(15, 10))
axes[0].imshow(ref_img, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
axes[2].imshow(tgt_aligned_homography, cmap="jet")
axes[2].set_xlabel(
    f"Registered (hompgraphy), ssim:{np.round(ssim(ref_img, tgt_aligned_homography), 3)}"
)
axes[3].imshow(tgt_aligned_affine, cmap="jet")
axes[3].set_xlabel(
    f"Registered (affine), ssim:{np.round(ssim(ref_img, tgt_aligned_affine), 3)}"
)

##### Phase Correlation

In [70]:
window = cv.createHanningWindow(ref_img_ds.shape, cv.CV_32F)
# window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_img_ds), np.float32(ref_img_ds), window
)
tgt_shifted_ds = warp_affine_dataset(
    tgt_img_ds, translation_x=shift_x, translation_y=shift_y
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_shifted_ds, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img_ds, tgt_shifted_ds), 3)}")

In [75]:
# window = cv.createHanningWindow(ref_img.shape, cv.CV_32F)
window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_img), np.float32(ref_img), window
)
tgt_shifted = warp_affine_dataset(tgt_img, translation_x=shift_x, translation_y=shift_y)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
axes[2].imshow(tgt_shifted, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img, tgt_shifted), 3)}")

##### Optical flow

In [111]:
# params for ShiTomasi corner detection
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7,
)

# Parameters for lucas kanade optical flow
number_of_iterations = 10
termination_eps = 1e-5
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(
        cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
        number_of_iterations,
        termination_eps,
    ),
)

In [112]:
p0 = cv.goodFeaturesToTrack(ref_img_ds, mask=None, **feature_params)
p1, st, err = cv.calcOpticalFlowPyrLK(ref_img_ds, tgt_img_ds, p0, None, **lk_params)
ref_good = p0[st == 1].astype(np.float32)
tgt_good = p1[st == 1].astype(np.float32)
warp_matrix = cv.findHomography(ref_good, tgt_good)[0]
tgt_aligned_ds = cv.warpPerspective(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_aligned_ds, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img_ds, tgt_aligned_ds), 3)}")

In [ ]:
p0 = cv.goodFeaturesToTrack(ref_img, mask=None, **feature_params)
p1, st, err = cv.calcOpticalFlowPyrLK(ref_img, tgt_img, p0, None, **lk_params)
ref_good = p0[st == 1].astype(np.float32)
tgt_good = p1[st == 1].astype(np.float32)
warp_matrix = cv.findHomography(ref_good, tgt_good)[0]
tgt_aligned = cv.warpPerspective(
    tgt_img,
    warp_matrix,
    (tgt_img.shape[1], tgt_img.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
axes[2].imshow(tgt_aligned, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img, tgt_aligned), 3)}")

#### Coregister S1 SLC (TRC) and S2 L1C 

In [ ]:
ref_scene = "../data/outputs/AROSICS/L1C_SLC_AU_pair/ref.tif"
tgt_scene = "../data/outputs/AROSICS/L1C_SLC_AU_pair/tgt.tif"

downsample_dataset(ref_scene, 0.1, "../data/outputs/OpenCV/L1C_SLC_AU/ref_ds.tif")
_ = downsample_dataset(tgt_scene, 0.1, "../data/outputs/OpenCV/L1C_SLC_AU/tgt_ds.tif")

In [76]:
ref_scene = "../data/outputs/OpenCV/L1C_SLC_AU/ref_ds.tif"
tgt_scene = "../data/outputs/OpenCV/L1C_SLC_AU/tgt_ds.tif"

In [ ]:
coords, warps = find_overlap(ref_scene, tgt_scene, True, True)

In [78]:
ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_scene).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img = cv.cvtColor(
    flip_img(rasterio.open(tgt_scene).read().copy()), cv.COLOR_BGR2GRAY
)

ref_warp = cv.cvtColor(warps[2], cv.COLOR_BGR2GRAY)
tgt_warp = cv.cvtColor(warps[3], cv.COLOR_BGR2GRAY)

##### ECC

In [6]:
warp_mode = cv.MOTION_HOMOGRAPHY
warp_matrix = np.eye(3, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_warp, tgt_warp, warp_matrix, warp_mode, criteria
)

tgt_aligned_homography_ds = cv.warpPerspective(
    tgt_warp,
    warp_matrix,
    (tgt_warp.shape[1], tgt_warp.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [7]:
warp_mode = cv.MOTION_AFFINE
warp_matrix = np.eye(2, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_warp, tgt_warp, warp_matrix, warp_mode, criteria
)

tgt_aligned_affine_ds = cv.warpAffine(
    tgt_warp,
    warp_matrix,
    (tgt_warp.shape[1], tgt_warp.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_aligned_homography_ds]), cmap="gray")
axes[1].set_xlabel(
    f"Registered (hompgraphy), ssim:{np.round(ssim(ref_warp, tgt_aligned_homography_ds), 3)}"
)
axes[2].imshow(simple_mosaic([ref_warp, tgt_aligned_affine_ds]), cmap="gray")
axes[2].set_xlabel(
    f"Registered (affine), ssim:{np.round(ssim(ref_warp, tgt_aligned_affine_ds), 3)}"
)

##### Phase correlation

In [114]:
# window = cv.createHanningWindow((ref_warp.shape[1], ref_warp.shape[0]), cv.CV_32F)
window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_warp), np.float32(ref_warp), window
)
tgt_shifted = warp_affine_dataset(
    tgt_warp, translation_x=shift_x, translation_y=shift_y
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_shifted]), cmap="gray")
axes[1].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_warp, tgt_shifted), 3)}")

##### Optical Flow

In [117]:
# params for ShiTomasi corner detection
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7,
)

# Parameters for lucas kanade optical flow
number_of_iterations = 10
termination_eps = 1e-5
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(
        cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
        number_of_iterations,
        termination_eps,
    ),
)

In [118]:
p0 = cv.goodFeaturesToTrack(ref_warp, mask=None, **feature_params)
p1, st, err = cv.calcOpticalFlowPyrLK(ref_warp, tgt_warp, p0, None, **lk_params)
ref_good = p0[st == 1].astype(np.float32)
tgt_good = p1[st == 1].astype(np.float32)
warp_matrix = cv.findHomography(ref_good, tgt_good)[0]
tgt_aligned_ds = cv.warpPerspective(
    tgt_warp,
    warp_matrix,
    (tgt_warp.shape[1], tgt_warp.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_aligned_ds]), cmap="gray")
axes[1].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_warp, tgt_aligned_ds), 3)}")